In [1]:
import pandas as pd
import numpy as np
import urllib.request
# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [6]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# !pip install geopy

print('Libraries imported.')

Libraries imported.


In [4]:
# From Part 1:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(wiki)
soup = BeautifulSoup(page)
# print(soup.prettify())
soup.title.string
data_table = soup.find('table')
# data_table
table_row = data_table.find_all('tr')

# Obtain the data from each row
df_data = []
for row in table_row:
#     print(row)
    table_data = row.find_all('td')
    data = [i.text.rstrip() for i in table_data]
    df_data.append(data)

# df_data
del df_data[0]
# df_data
labels = ['Postcode', 'Borough', 'Neighborhood']
df_table = pd.DataFrame(columns = labels)
df_table
df_table = pd.DataFrame.from_records(df_data, columns=labels)
# df_table.head()

# Remove the cells with 'Not assigned' value
df_table = df_table[df_table.Borough != 'Not assigned']
# df_table.head()

# Part 2: Obtain the geographical coordinates csv file
df_geocsv = pd.read_csv('http://cocl.us/Geospatial_data').set_index('Postal Code')
df_geocsv.head(10)

# Rename the "Postcode" as "Postal Code"
df_table.rename(columns={"Postcode":"Postal Code"}, inplace=True)
df_table.set_index('Postal Code')
# df_table.head()

df_cangeodata = pd.merge(df_table, df_geocsv, on='Postal Code')
df_cangeodata

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [18]:
df_togeodata = df_cangeodata[df_cangeodata['Borough'].str.contains('Toronto', regex=False)]
df_togeodata.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [19]:
# create map of Toronto using latitude and longitude values
latitude = 43.651070
longitude = -79.347015

map_toronto = folium.Map(location=[latitude, longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df_togeodata['Latitude'], df_togeodata['Longitude'], df_togeodata['Borough'], df_togeodata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [20]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = df_togeodata.drop(['Postal Code', 'Borough', 'Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1], dtype=int32)

In [21]:
df_togeodata.insert(0, 'Cluster Labels', kmeans.labels_)
df_togeodata.head()

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_togeodata['Latitude'], df_togeodata['Longitude'], df_togeodata['Neighborhood'], df_togeodata['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters